Tensorflow is a software framework for building and deploying machine learning models.Tensorflow can only work with NVIDIA-brand GPUs
Tensorflows core execution engine is written in c++  for faster speed.Python is best supported and easiest language to use with 
tensorflow.

keras is high level Tensorflow wrapper for building neural networks with only few lines of code.

Tensors are multi dimmensional arrays


Tensorflow session -an object that runs operations on the computation graph and tracks the state of each node in the graph

Tensorflow cutom pipeline -It can scale to infinitely large data sets and take advantage of parallel processing.

In [23]:
import os
#import tensorflow as tf

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
tf.__version__

'2.2.0'

In [3]:
#turn off tensorflow warning message in program output
os.environ['TF_CPP_MIN_LOG_LEVEL']="2"

In [24]:
#Define computational graph
X=tf.placeholder(tf.float32,name="X")
Y=tf.placeholder(tf.float32,name="Y")


addition=tf.add(X,Y,name="addition")

In [25]:
#create session
with tf.Session() as session:
    result=session.run(addition,feed_dict={X:[[1,2,10],[2,4,6]],Y:[[4,2,10],[1,2,3]]})
    print(result)
    

[[ 5.  4. 20.]
 [ 3.  6.  9.]]


###### Load data set
1.simple with pandas dataframe
2.pipeline - when large data set(particularly when working with image)
3. 

In [26]:
import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior() 

In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [28]:
import os
print(os.getcwd())

C:\Users\prajaktab2\A-prajakta\LEARNINGS_NLP\Tensorboard\Ex_Files_TensorFlow\Exercise Files


In [29]:
data=pd.read_csv(r"./03/sales_data_training.csv",dtype=float)

In [30]:
data.head(3)

,critic_rating,is_action,is_exclusive_to_us,is_portable,is_role_playing,is_sequel,is_sports,suitable_for_kids,total_earnings,unit_price
0,3.5,1.0,0.0,1.0,0.0,1.0,0.0,0.0,132717.0,59.99
1,4.5,0.0,0.0,0.0,0.0,1.0,1.0,0.0,83407.0,49.99
2,3.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,62423.0,49.99


In [31]:
#pull out columns for X (data to train with ) and Y (value to predict)
X_training=data.drop(columns={"total_earnings"}).values
Y_training=data[['total_earnings']].values

In [32]:
data_test=pd.read_csv("./03/sales_data_test.csv",dtype=float)

In [33]:
data_test.head(2)

,critic_rating,is_action,is_exclusive_to_us,is_portable,is_role_playing,is_sequel,is_sports,suitable_for_kids,total_earnings,unit_price
0,3.5,1.0,1.0,1.0,0.0,1.0,0.0,1.0,247537.0,59.99
1,2.5,0.0,0.0,0.0,1.0,1.0,0.0,0.0,73960.0,59.99


In [34]:
X_test=data_test.drop(columns={"total_earnings"}).values
Y_test=data_test[['total_earnings']].values

All data needs to be scaled on smaoll scale(0 to 1) for neural network to work well.Create scalers for input and outputs

In [35]:
X_scaler=MinMaxScaler(feature_range=(0,1))
Y_scaler=MinMaxScaler(feature_range=(0,1))

In [36]:
X_scaled_training=X_scaler.fit_transform(X_training)
Y_scaled_training=Y_scaler.fit_transform(Y_training)

It is very important that the training and test data are scaled with same scaler so we will use just transform method
instead of fit_transform

In [37]:
X_scaled_testing=X_scaler.transform(X_test)
Y_scaled_testing=Y_scaler.transform(Y_test)

In [38]:
X_scaled_testing.shape

(400, 9)

In [39]:
Y_scaled_testing.shape

(400, 1)

In [40]:
#### Define model parameters
learning_rate=0.001
epochs=100
display_step=5

In [41]:
# Define how many input and output parameters are in our neural network
number_of_inputs=9
number_of_outputs=1

In [42]:
#Define how many neurons we want in each layer of neural network
layer_1_nodes=50
layer_2_nodes=100
layer_3_nodes=50

In [43]:
with tf.variable_scope('input'):
    X=tf.placeholder(tf.float32,shape=(None,number_of_inputs))

In [44]:
with tf.variable_scope('layer_1'):
    weights=tf.get_variable(name="weights1",shape=[number_of_inputs,layer_1_nodes],initializer=tf.zeros_initializer())
    biases=tf.get_variable(name="biases1",shape=[layer_1_nodes],initializer=tf.zeros_initializer())
    layer_1_output=tf.nn.relu(tf.matmul(X,weights)+biases)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [45]:
with tf.variable_scope('layer_2'):
    weights=tf.get_variable(name="weights2",shape=[layer_1_nodes,layer_2_nodes],initializer=tf.zeros_initializer())
    biases=tf.get_variable(name="biases2",shape=[layer_2_nodes],initializer=tf.zeros_initializer())
    layer_2_output=tf.nn.relu(tf.matmul(layer_1_output,weights)+biases)

In [46]:
with tf.variable_scope('layer_3'):
    weights=tf.get_variable(name="weights3",shape=[layer_2_nodes,layer_3_nodes],initializer=tf.zeros_initializer())
    biases=tf.get_variable(name="biases3",shape=[layer_3_nodes],initializer=tf.zeros_initializer())
    layer_3_output=tf.nn.relu(tf.matmul(layer_2_output,weights)+biases)

In [47]:
with tf.variable_scope('output'):
    
    weights=tf.get_variable(name="weights4",shape=[layer_3_nodes,number_of_outputs],initializer=tf.zeros_initializer())
    biases=tf.get_variable(name="biases4",shape=[number_of_outputs],initializer=tf.zeros_initializer())
    prediction=tf.nn.relu(tf.matmul(layer_3_output,weights)+biases)    
    

#Define the cost function for neual network

In [48]:
with tf.variable_scope('cost'):
    Y=tf.placeholder(tf.float32,shape=(None,1))
    cost=tf.reduce_mean(tf.squared_difference(prediction,Y))

In [49]:
#Define optimizer to optimize neural network
with tf.variable_scope('train'):
    optimizer=tf.train.AdamOptimizer(learning_rate).minimize(cost)


#logging information to tensorboard

In [50]:
with tf.variable_scope('logging'):
    tf.summary.scalar("current_cost",cost)
    summary=tf.summary.merge_all()

To run any operation on tensorflow graph we need to create session

In [55]:
with tf.Session() as session:
    #run the global variable initializer to initialize all variables and layers of the graph
    session.run(tf.global_variables_initializer())
    
    #create log file writers to record training progress
    # we will store training and testing log data seperatly
    training_Writer=tf.compat.v1.summary.FileWriter("./logs/training",session.graph)
    testing_writer=tf.compat.v1.summary.FileWriter("./logs/testing",session.graph)
    #Run the optimizer over and over to train the network
    #one epoch is one full run through the training data set
    
    for epoch in range(epochs):
        #Feed in the training data and do one step of neural network training data set
        session.run(optimizer,feed_dict={X:X_scaled_training,Y:Y_scaled_training})
        #print current training status to console
        print("Training pass :{}".format(epoch))
        
        if epoch %5 ==0:
            training_cost,training_summary=session.run([cost,summary],feed_dict={X:X_scaled_training,Y:Y_scaled_training})
            testing_cost,testing_summary=session.run([cost,summary],feed_dict={X:X_scaled_testing,Y:Y_scaled_testing})
            print(epoch,training_cost,testing_cost)
            
            #write current status to log file
            training_Writer.add_summary(training_summary,epoch)
            testing_writer.add_summary(testing_summary,epoch)
    
    #training is now complete
    print("Training is completed")
    final_training_cost=session.run(cost,feed_dict={X:X_scaled_training,Y:Y_scaled_training})
    final_testing_cost=session.run(cost,feed_dict={X:X_scaled_testing,Y:Y_scaled_testing})
    
    print("Final training cost: {}".format(final_training_cost))
    print("Final testing cost : {}".format(final_testing_cost))

Training pass :0
0 0.113659315 0.12273928
Training pass :1
Training pass :2
Training pass :3
Training pass :4
Training pass :5
5 0.113659315 0.12273928
Training pass :6
Training pass :7
Training pass :8
Training pass :9
Training pass :10
10 0.113659315 0.12273928
Training pass :11
Training pass :12
Training pass :13
Training pass :14
Training pass :15
15 0.113659315 0.12273928
Training pass :16
Training pass :17
Training pass :18
Training pass :19
Training pass :20
20 0.113659315 0.12273928
Training pass :21
Training pass :22
Training pass :23
Training pass :24
Training pass :25
25 0.113659315 0.12273928
Training pass :26
Training pass :27
Training pass :28
Training pass :29
Training pass :30
30 0.113659315 0.12273928
Training pass :31
Training pass :32
Training pass :33
Training pass :34
Training pass :35
35 0.113659315 0.12273928
Training pass :36
Training pass :37
Training pass :38
Training pass :39
Training pass :40
40 0.113659315 0.12273928
Training pass :41
Training pass :42
Trai

In [56]:
%load_ext tensorboard 

In [59]:
os.getcwd()

'C:\\Users\\prajaktab2\\A-prajakta\\LEARNINGS_NLP\\Tensorboard\\Ex_Files_TensorFlow\\Exercise Files'

In [68]:
!kill 31124

'kill' is not recognized as an internal or external command,
operable program or batch file.


In [70]:
%tensorboard --logdir ./logs/training #!kill 36432

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 41972.

#Saver.save
#save.restore

Checkpoint file -a file that contains the state of a trained machine learning model